In [1]:
!pip install -q accelerate peft bitsandbytes transformers trl

In [2]:
import os
import torch
from datasets import Dataset, DatasetDict
from torch.nn import DataParallel
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer
import io
from huggingface_hub import notebook_login
import pandas as pd

2024-06-21 23:26:58.705419: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-21 23:26:58.705527: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-21 23:26:58.835121: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
notebook_login()

In [3]:

df = pd.read_excel('/kaggle/input/fine-tuning/Dataset_creation.xlsx')
df["formatted_instruction"] = df.apply(lambda x: f"<s>[INST] {x['Instructions']} [/INST] {x['Responses']} </s>", axis=1)
#df = df.drop(columns=df.columns[:2])
df = df.drop(columns=['Instructions', 'Responses'])
df.to_csv("conversational_genAI.csv", index=False)
#created_dataset = Dataset.from_pandas(df)
#print(created_dataset)


df_read = pd.read_csv("/kaggle/working/conversational_genAI.csv")

# Ensure the key 'formatted_instruction' exists in the DataFrame
if "formatted_instruction" not in df_read.columns:
    raise ValueError("The key 'formatted_instruction' does not exist in the DataFrame")

# Convert DataFrame to Hugging Face Dataset
dataset = Dataset.from_pandas(df_read)

# Create a DatasetDict with the train split
dataset_dict = DatasetDict({
    "train": dataset
})
#device = "cuda:0" if torch.cuda.is_available() else "cpu"
# model name
model_name = "meta-llama/Llama-2-7b-hf"
new_model = "llama-2-7b-user-manuals-new"

max_seq_length = 128                    # Maximum sequence length to use
packing = False                        # Pack multiple short examples in the same input sequence to increase efficiency
device_map = {"": 0}                   # Load the entire model on the GPU 0


# Tokenize the dataset (example tokenization step)
def tokenize_function(examples):
    return tokenizer(examples["formatted_instruction"], padding="max_length", truncation=True, max_length=max_seq_length)
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right" # Fix weird overflow issue with fp16 training
tokenized_dataset = dataset_dict.map(tokenize_function, batched=True)

#dataset_name = "/content/conversational_genAI.csv"
#def tokenize(example):
#    return tokenizer(example['formatted_instruction'], truncation=True, padding='max_length',max_length=max_seq_length)


#tokenized_dataset = created_dataset.map(tokenize, batched=True, remove_columns=created_dataset.column_names)
#print(tokenized_dataset[0])

Map:   0%|          | 0/138 [00:00<?, ? examples/s]

In [7]:
#-----QLora parameters------#
lora_r = 16            # attention mechanism
lora_alpha = 64        # scaling purpose
lora_dropout = 0.1     # dropout probaility layer

#----Bits and bytes parameters-------#
load_4bit=True                      # Activate 4-bit precision base model loading
bnb_4bit_quant_type="nf4"           # Quantization type (fp4 or nf4)
bnb_4bit_compute_dtype=torch.float16    # Compute dtype for 4-bit base models
#compute_dtype = getattr(torch, bnb_4bit_compute_dtype)
use_nested_quant = False

#----Training arguments parameters-----#
output_dir = "./results"               # output directory
num_train_epochs = 10                   # traning epochs
fp16 = False                           # Enable fp16/bf16 training (set bf16 to True with an A100)
bf16 = False
per_device_train_batch_size = 4        # Batch size per GPU for training
per_device_eval_batch_size = 4         # Batch size per GPU for evaluation
gradient_accumulation_steps = 1        # Number of update steps to accumulate the gradients for
gradient_checkpointing = True
max_grad_norm = 0.3                    # Maximum gradient normal (gradient clipping)
learning_rate = 2e-4
weight_decay = 0.01                   # Weight decay to apply to all layers except bias/LayerNorm weights
optim = "paged_adamw_32bit"            # optimizer
lr_scheduler_type = "cosine"           # Learning rate scheduler
max_steps = -1                         # Number of training steps (overrides num_train_epochs)
warmup_ratio = 0.03                    # Ratio of steps for a linear warmup (from 0 to learning rate)
group_by_length = True                 # Group sequences into batches with same length. Saves memory and speeds up training considerably
save_steps = 0                         # Save checkpoint every X updates steps
logging_steps = 5                     # Log every X updates steps




In [24]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=load_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=bnb_4bit_compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

# Load base model
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map=device_map #'auto'
)
model.config.use_cache = False
model.config.pretraining_tp = 1

# Load LLaMA tokenizer
#tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
#tokenizer.pad_token = tokenizer.eos_token
#tokenizer.padding_side = "right" # Fix weird overflow issue with fp16 training

# Move model to GPU and wrap with DataParallel if multiple GPUs are available


# Load LoRA configuration
peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
)
# peft_config = LoraConfig(
#     lora_alpha=lora_alpha,
#     lora_dropout=lora_dropout,
#     r=lora_r,
#     target_modules=["q_proj", "v_proj"],  # Example for Llama-2, adjust as needed for your model
#     bias="none",
#     task_type="CAUSAL_LM",
# )
# Set training parameters
training_arguments = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    fp16=fp16,
    bf16=bf16,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type,
    report_to="tensorboard"
)

# df_read = pd.read_csv("/content/conversational_genAI.csv")
# Ensure the key exists in the DataFrame
if "formatted_instruction" not in df_read.columns:
    raise ValueError("The key 'formatted_instruction' does not exist in the DataFrame")

# Assuming you want to create a DatasetDict with the key "formatted_instruction"
dataset_dict = DatasetDict({"formatted_instruction": dataset})

# Set supervised fine-tuning parameters
trainer = SFTTrainer(
    model=model,
    train_dataset=tokenized_dataset["train"],
    peft_config=peft_config,
    dataset_text_field="formatted_instruction",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=packing,
)

trainer.train()
if isinstance(trainer.model, DataParallel):
    trainer.model.module.save_pretrained(new_model)
else:
    trainer.model.save_pretrained(new_model)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/opt/conda/lib/python3.10/site-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': dataset_text_field, max_seq_length. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1965: FutureWarning: `--push_to_hub_token` is deprecated and will be removed in version 5 of 🤗 Transformers. Use `--hub_token` instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1965: FutureWarning: `--push_to_hub_token` is deprecated and will be removed in version 5 of 🤗 Transformers. Use `--hub_token` instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:269: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the

Step,Training Loss
5,3.353900
10,3.066800
15,2.769000
20,2.490300
25,2.476000
30,2.243700
35,2.071500
40,1.891400
45,2.150400
50,1.882900


/opt/conda/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [26]:
# Ignore warnings
# logging.set_verbosity(logging.CRITICAL)

# Run text generation pipeline with our next model
prompt = "What components make up a touch sensitive display/console in laser system?"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200)
result = pipe(f"<s>[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


<s>[INST] What components make up a touch sensitive display/console in laser system? [/INST] Touch displays consist of the display itself and a superimposed touch-sensitive coating. 1. Display - shows error messages and laser images. 2. Touch-sensitive coating - enables user inputs via swiping etc. 3. Bezel - outer frame of the display, holds it in place in the console. 4. Touch-sensitive button - used to confirm entries in the client interface. 5. Head set connector - for connecting a head set. 6. LEDs - display status signals. 7. Edge protector - safeguards the display against damage in case of contact with an edge. 8. Touch-sensitive function button - used to execute input functions (e g. save, select, etc.). 9. Tilt safety switch - switches off the laser system in case


In [8]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel

# Load the base model
base_model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-2-7b-hf")

# Load the adapter model
adapter_model_id = "/kaggle/working/10-epoch-mode/kaggle/working/llama-2-7b-user-manuals"
loaded_model = PeftModel.from_pretrained(base_model, adapter_model_id, is_trainable=False)

config.json:   0%|          | 0.00/609 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

In [9]:
del loaded_model

In [10]:
!zip -r file.zip /kaggle/working/llama-2-7b-user-manuals

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


	zip warning: name not matched: /kaggle/working/llama-2-7b-user-manuals

zip error: Nothing to do! (try: zip -r file.zip . -i /kaggle/working/llama-2-7b-user-manuals)


In [12]:
# del tokenizer
del model
del pipe
del trainer
import gc
gc.collect()
gc.collect()

NameError: name 'pipe' is not defined

In [ ]:
import logging
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel

# Load the base model
base_model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-2-7b-hf")

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-hf")

# Load the adapter model
adapter_model_id = "/kaggle/working/10-epoch-mode/kaggle/working/llama-2-7b-user-manuals"
model = PeftModel.from_pretrained(base_model, adapter_model_id, is_trainable=False)

# Set logging level to ignore warnings
logging.getLogger().setLevel(logging.CRITICAL)

# Define the prompt
prompt = "What components make up a touch sensitive display/console in laser system?"

# Create the text generation pipeline
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200)

# Generate text
result = pipe(f"<s>[INST] {prompt} [/INST]")

# Print the generated text
print(result[0]['generated_text'])

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

# hi masti ahe 

In [6]:
# Load the base model
base_model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-2-7b-hf")

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-hf")

# Load the adapter model
adapter_model_id = "/kaggle/working/10-epoch-mode/kaggle/working/llama-2-7b-user-manuals"
model = PeftModel.from_pretrained(base_model, adapter_model_id, is_trainable=False)

# model = PeftModel.from_pretrained(base_model, adapter_model_id, is_trainable=False)
model = model.merge_and_unload()

# Reload tokenizer to save it
tokenizer = AutoTokenizer.from_pretrained(base_model, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

model-00001-of-00002.safetensors:  21%|##1       | 2.12G/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/776 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

OSError: Incorrect path_or_model_id: 'LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (up_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear(in_features=11008, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (

# yachya khali snehal ne chukavlay

In [5]:
import shutil

source_dir = "/kaggle/input/10-epoch-model" 

# Destination directory
destination_dir = "/kaggle/working/10-epoch-mode"

shutil.copytree(source_dir, destination_dir)

'/kaggle/working/10-epoch-mode'

In [ ]:
%load_ext tensorboard
%tensorboard --logdir results/runs

In [9]:
import gc
gc.collect()

197

In [9]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel
import json
import torch

#device = "cuda:0" if torch.cuda.is_available() else "cpu"
save_directory = "/kaggle/working/trained-model-1-original"
import os
os.makedirs(save_directory, exist_ok=True)
#-----QLora parameters------#
lora_r = 16            # attention mechanism
lora_alpha = 64        # scaling purpose
lora_dropout = 0.1     # dropout probability layer

adapter_config = {
    "lora_alpha": lora_alpha,
    "lora_dropout": lora_dropout,
    "r": lora_r,
    "target_modules": ["q_proj", "v_proj"],  # Adjust as needed
    "bias": "none",
    "task_type": "CAUSAL_LM",
    "peft_type": "LORA"  # Add this line to specify the PEFT type
}

# Save adapter configuration to the same directory as the adapter model weights
with open(f"{ save_directory}/adapter_config.json", "w") as f:
    json.dump(adapter_config, f)

# Load base model
model_name = "/meta-llama/Llama-2-7b-hf"
base_model = AutoModelForCausalLM.from_pretrained(model_name)

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Load adapter model weights and configuration
adapter_model = PeftModel.from_pretrained(base_model, save_directory)

# Now the adapter model can be used for inference or further training
# Example usage for inference:
input_text = "What are the parameters for pulse time in pulsed operation?"
inputs = tokenizer(input_text, return_tensors="pt")
outputs = adapter_model.generate(**inputs)
generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

print(generated_text)


OSError: Incorrect path_or_model_id: '/meta-llama/Llama-2-7b-hf'. Please provide either the path to a local folder or the repo_id of a model on the Hub.

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel
import json
import torch

# Check if GPU is available
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)
# Directory to save the model
save_directory = "/kaggle/working/llama-2-7b-user-manuals"
import os
os.makedirs(save_directory, exist_ok=True)

# QLoRA parameters
lora_r = 16            # Attention mechanism
lora_alpha = 64        # Scaling purpose
lora_dropout = 0.1     # Dropout probability layer

adapter_config = {
    "lora_alpha": lora_alpha,
    "lora_dropout": lora_dropout,
    "r": lora_r,
    "target_modules": ["q_proj", "v_proj"],  # Adjust as needed
    "bias": "none",
    "task_type": "CAUSAL_LM",
    "peft_type": "LORA"  # Add this line to specify the PEFT type
}

# Save adapter configuration to the same directory as the adapter model weights
with open(f"{save_directory}/adapter_config.json", "w") as f:
    json.dump(adapter_config, f)

# Load base model
model_name = "meta-llama/Llama-2-7b-hf"
base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map='auto'  # Automatically distribute across available GPUs
)

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Load adapter model weights and configuration
adapter_model = PeftModel.from_pretrained(base_model, save_directory)
adapter_model.to(device)  # Move the model to the GPU

# Now the adapter model can be used for inference or further training
# Example usage for inference:
input_text = "What are the parameters for pulse time in pulsed operation?"
inputs = tokenizer(input_text, return_tensors="pt").to(device)  # Move inputs to GPU
outputs = adapter_model.generate(**inputs)
generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

print(generated_text)


cuda


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel, PeftConfig

import json

# Directory paths
input_directory = "/kaggle/input/trained-model-1"
save_directory = "/kaggle/working/llama-2-7b-user-manuals"

# Read adapter configuration from the read-only input directory
adapter_config_path = f"{input_directory}/adapter_config.json"
with open(adapter_config_path, "r") as f:
    adapter_config = json.load(f)

#-----QLora parameters from the config------#
lora_r = adapter_config["r"]
lora_alpha = adapter_config["lora_alpha"]
lora_dropout = adapter_config["lora_dropout"]
peft_type = adapter_config["peft_type"]  # Should be "lora" or other supported types

# Ensure the save directory exists for any other outputs
import os
os.makedirs(save_directory, exist_ok=True)

# Load base model
model_name = "meta-llama/Llama-2-7b-hf"
base_model = AutoModelForCausalLM.from_pretrained(model_name)

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Load adapter model weights and configuration
adapter_model = PeftModel.from_pretrained(base_model, input_directory)

# Now the adapter model can be used for inference or further training
# Example usage for inference:
input_text = "What are the parameters for pulse time in pulsed operation?"
inputs = tokenizer(input_text, return_tensors="pt")
outputs = adapter_model.generate(**inputs)
generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

print(generated_text)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

KeyboardInterrupt: 